In [1]:
from opentelemetry import trace
from opentelemetry.exporter.cloud_trace import CloudTraceSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.trace import Link
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [2]:

tracer_provider = TracerProvider()
cloud_trace_exporter = CloudTraceSpanExporter()
tracer_provider.add_span_processor(
    # BatchSpanProcessor buffers spans and sends them in batches in a
    # background thread. The default parameters are sensible, but can be
    # tweaked to optimize your performance
    BatchSpanProcessor(cloud_trace_exporter)
)
trace.set_tracer_provider(tracer_provider)

tracer = trace.get_tracer(__name__)

In [3]:
from vertexai.language_models import TextEmbeddingModel

def text_embedding(tracer_object, text: str = "What is life?") -> list:
    """Text embedding with a Large Language Model."""
    with tracer.start_as_current_span("text_embeddings", links=[Link(tracer_object.context)]) as text_embeddings:
        model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
        embeddings = model.get_embeddings([text])
        vector = embeddings[0].values
        #print(f"Length of Embedding Vector: {len(vector)}")
    return vector

In [4]:
db_user = os.environ['DB_USER']
db_pass = os.environ['DB_PASSWORD']
db_host = os.environ['DB_HOST']
db_name = 'postgres'
db_port = '5432'  # Default PostgreSQL port
table_name = 'embeddings_sample'  # Table where you want to store the embeddings

In [5]:
import json
import psycopg2

def get_documents(tracer_object, vector):
    with tracer.start_as_current_span("get_documents", links=[Link(tracer_object.context)]) as get_documents:
        

        conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pass, host=db_host, port=db_port)

        # Assuming `embeddings` is a list of embedding vectors you received from the textembedding service
        # And `texts` is the list of texts corresponding to each embedding
        #for text, embedding in zip(texts, embeddings):
            # Convert the embedding to a format suitable for storage, e.g., a string
            #embedding_str = f"[{','.join(map(str, str(embedding)))}]"
        #print(embeddings)
            # Insert into database
        results = None
        with conn.cursor() as cur:
            cur.execute(f"""SELECT DISTINCT content, 1 - (embedding <=> '{vector}') AS similarity FROM embeddings_sample
                         WHERE 1 - (embedding <=> '{vector}') > .72
                         ORDER BY similarity DESC
                         LIMIT 10""")
            results = cur.fetchall()

        conn.close()
                    
    return results

In [13]:
# langchain_integration.py

def get_billing_data_for_project(tracer_object, project_id):
    
    with tracer.start_as_current_span("get_cost", links=[Link(tracer_object.context)]) as get_cost:
    
        conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pass, host=db_host, port=db_port)

        # Query the database for billing data related to the specified project
        query = """
        SELECT service_name, SUM(cost) as total_cost
        FROM gcp_billing_data
        WHERE project_id = %s
        GROUP BY service_name
        """
        with conn.cursor() as cursor:
            cursor.execute(query, (project_id,))
            results = cursor.fetchall()
        conn.close()
    return results


In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain
from IPython.display import display, Markdown
from langchain.llms import VertexAI

def llm_request(tracer_object, question):
    
    with tracer.start_as_current_span("main_llm", links=[Link(tracer_object.context)]) as main_llm:
        
        model = VertexAI(model_name="gemini-pro", temperature=0.5, max_output_tokens=2048)
        
        # ==== Get project id ====
        project_template = """identify the google cloud project id in the following text: 
        {question}
        """
        
        project_prompt = PromptTemplate(template=project_template, input_variables=["question"])
        
        project_chain = LLMChain(llm=model,
                                  prompt=project_prompt,
                                  output_key="project_id",
                                 )

        
        with tracer.start_as_current_span("llm_project_id", links=[Link(tracer_object.context)]) as llm_project_id:
            
            tracer_object.set_attribute("prompt length", len('\n'.join(question)))

            project_answer = project_chain({
                "question": question,
                })
            
            
        
        # === Retrieve data for the project ===
        
        #project_data = get_billing_data_for_project(tracer_object, project_answer['project_id'])
        
        
        # === Summarize the data ====
        summary_template = """in a moment I will provide you with a set of text. Summarize the main points of these texts and include any expertise DoiT can provide.
        
        {chunks}
        """
        
        summary_prompt = PromptTemplate(template=summary_template, input_variables=["chunks"])

        summary_chain = LLMChain(llm=model,
                                  prompt=summary_prompt,
                                  output_key="context",
                                 )
        
        #with tracer.start_as_current_span("llm_summary_invoke", links=[Link(tracer_object.context)]) as llm_summary_invoke:
            
            #tracer_object.set_attribute("prompt length", len('\n'.join(matches)))

            #summary_answer = summary_chain.({
            #    "chunks": matches,
            #    })
            
        
        # === Retrieve context data ====
        vector = text_embedding(tracer_object, "how can cost be manage in google cloud?")

        matches = get_documents(tracer_object, vector)

        matches = '\n'.join([match[0] for match in matches])
        
        
            
        #display(Markdown(summary_answer))
        
        # === Retrieve analysis ===
        
        template = """This is information on how to do cost analysis
        
                    {context}
        
                    In a moment I will give you data for the cost of service in a specific google project. Using the following information to help create a cost analysis:
                    
                    cost data for {project_id}
                    
                    {project_data}
                    
                    Question: {question}
                    """
    
    
        prompt = PromptTemplate(template=template, input_variables=["context", "project_id", "project_data", "question"])

        chain = LLMChain(llm=model,
                          prompt=prompt,
                          output_key="analysis",
                         )
        
        with tracer.start_as_current_span("llm_invoke", links=[Link(tracer_object.context)]) as llm_invoke:
            
            #tracer_object.set_attribute("prompt length", len(summary_answer) + len(question))

            overall_chain = SequentialChain(
                chains=[summary_chain, chain],
                input_variables=["question", "chunks", "project_data", "project_id"],
                output_variables=["context", "analysis"]
            )
            
            answer = overall_chain({"question": question,
                                    "chunks": matches,
                                    "project_data": get_billing_data_for_project(tracer_object, project_answer['project_id']),
                                    "project_id": project_answer['project_id']
                                   })

    return answer['analysis']




In [15]:
import decimal

class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super().default(o)

In [17]:

with tracer.start_as_current_span("qa_llm_evaluation") as llm_evaluation:
    query = "Analyze the cost of the project proj-webapp-3?"

    answer = llm_request(llm_evaluation, query)
    
    llm_evaluation.set_attribute("output length", len(answer))
    
    llm_evaluation.add_event(name="received output")

    display(Markdown(answer))
    
    llm_evaluation.add_event(name="display output")



**Cost Analysis for Project proj-webapp-3**

**Cost Allocation:**

The project's total cost is the sum of the individual service costs:

```
Total Cost = Cloud Storage + Cloud CDN + Cloud SQL + App Engine + Identity Platform + Cloud Load Balancing + Cloud Firestore + Cloud Pub/Sub + Cloud Functions + Compute Engine
```

```
Total Cost = 32.5000 + 41.9700 + 49.8100 + 9.0500 + 44.3600 + 19.3500 + 13.8200 + 32.0100 + 65.7300 + 39.9100
```

```
Total Cost = 348.5100
```

**Cost Optimization:**

* **Cloud Storage:** Consider using a more cost-effective storage class or optimizing object lifecycle management.
* **Cloud CDN:** Explore optimizing cache policies and reducing bandwidth usage.
* **Cloud SQL:** Review database usage patterns and consider using a smaller instance or optimizing database performance.
* **App Engine:** Analyze application performance and consider scaling down instances or optimizing code.
* **Identity Platform:** Review user authentication patterns and consider optimizing access controls.
* **Cloud Load Balancing:** Ensure that load balancers are configured optimally for traffic volume.
* **Cloud Firestore:** Optimize data structures and indexing to reduce read and write costs.
* **Cloud Pub/Sub:** Review message volume and consider using a more cost-effective pricing tier.
* **Cloud Functions:** Analyze function execution time and consider optimizing code or reducing function invocations.
* **Compute Engine:** Review instance usage patterns and consider using spot instances or preemptible VMs.

**Cost Governance:**

* Establish cost budgets and alerts to monitor spending and prevent overages.
* Implement automated cost allocation mechanisms to track costs by department or project.
* Regularly review cost reports and identify areas for further optimization.

**Cloud Cost Analytics:**

* Use tools like DoiT Cloud Analytics to analyze billing data, identify cost drivers, and track optimization efforts.
* Monitor cost trends over time and identify any anomalies or unexpected increases.

**External Expertise:**

* Consider consulting with cloud partners like DoiT to enhance cost management, analytics, and governance.
* Leverage their expertise in cost optimization, automated governance, and advanced hardware solutions.